In [1]:
import torch
import os
import copy
import random
import timm
import wandb
import cv2
import torchmetrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import pytorch_lightning as pl
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as lr_scheduler

from adamp import AdamP
from datetime import datetime, timezone, timedelta
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ipywidgets import interact

from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional import accuracy, f1_score, precision, recall
# from sklearn.metrics import accuracy_score, f1_score

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from pytorch_lightning.loggers import WandbLogger

In [14]:
# 프로젝트 경로
PROJECT_DIR = './apple/'
# os.chdir(PROJECT_DIR)

# 데이터 경로
DATA_DIR = os.path.join(PROJECT_DIR, 'DATA')
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
TRAIN_LABEL_DIR = os.path.join(DATA_DIR, 'train.csv')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')
TEST_LABEL_DIR = os.path.join(DATA_DIR, 'test.csv')
SAMPLE_DIR = os.path.join(DATA_DIR, 'sample_submission.csv')

In [15]:
# 파일 수 확인 
print(len(os.listdir(TRAIN_IMG_DIR)))
print(len(os.listdir(TEST_IMG_DIR)))

10000
5000


In [16]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [17]:
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
class TestDataset(Dataset):
    def __init__(self, img_folder, label_path):
        self.df = pd.read_csv(label_path, usecols=['img_id'], dtype={'a_type':str})
        # self.df = label_path
        self.img_folder = img_folder
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
        self.img_ids = list(self.df['img_id'])

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_folder, self.img_ids[index])
        img = Image.open(image_path)
        img = self.transform(img)
        img_ids = self.img_ids[index]

        return img, img_ids

In [19]:
class EfficientnetB4(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()

        self.model = timm.create_model('efficientnet_b4', pretrained=True)
        self.model.classifier = nn.Sequential(
        nn.Linear(in_features = 1792, out_features = 625),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(625,256),
        nn.ReLU(),
        nn.Linear(256, 4)
        )

        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output


    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/f1"}

        elif self.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}


    def training_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("train/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("train/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("train/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}


    def validation_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("val/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("val/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("val/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}

In [20]:
class InceptionV3(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = timm.create_model('inception_v3', pretrained=True, num_classes=4)
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output


    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/f1"}

        elif self.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}


    def training_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("train/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("train/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("train/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}


    def validation_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("val/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("val/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("val/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}

In [21]:
class SeResnext26(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = timm.create_model('seresnext26d_32x4d', pretrained=True, num_classes=4)
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output


    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/f1"}

        elif self.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}


    def training_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("train/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("train/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("train/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}


    def validation_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("val/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("val/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("val/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}

In [22]:
class SeResnext101(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = timm.create_model('seresnext101d_32x8d', pretrained=True, num_classes=4)
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler    

    def forward(self, x):
        output = self.model(x)
        return output


    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/f1"}

        elif self.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}


    def training_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("train/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("train/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("train/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}


    def validation_step(self, train_batch, batch_idx):
        image, label = train_batch

        outputs = self.model(image)
        loss = self.criterion(outputs, label)
        acc = accuracy(outputs, label)
        f1 = f1_score(outputs, label, num_classes=4, average="macro")
        self.log("val/acc", acc, on_epoch=True, on_step=True, prog_bar=True)
        self.log("val/loss", loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log("val/f1", f1, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)

        return {"acc":acc, "loss":loss, "f1-score":f1}

In [ ]:
model1 = EfficientnetB4()
model2 = InceptionV3()
model3 = SeResnext26()
# model4 = SeResnext101()

model1 = model1.load_from_checkpoint('./apple/Models/effi.ckpt')
model2 = model2.load_from_checkpoint('./apple/Models/inception.ckpt')
model3 = model3.load_from_checkpoint('./apple/Models/res26.ckpt')
# model4 = model4.load_from_checkpoint('./apple/Models/res101.ckpt')

model1.to(device)
model2.to(device)
model3.to(device)
# model4.to(device)

model1.eval()
model2.eval()
model3.eval()
# model4.eval()

In [27]:
test_dataset = TestDataset(img_folder = TEST_IMG_DIR, label_path = os.path.join(TEST_LABEL_DIR))

# DataLoader
test_dataloader = DataLoader(dataset = test_dataset,
                              batch_size = 1,
                              num_workers = 0,
                              shuffle = False,
                              drop_last = False)

In [32]:
# 예측 진행
y_preds = []
img_ids = []

for batch_index, (x, img_id) in enumerate(tqdm(test_dataloader)):
    x = x.to(device, dtype=torch.float)
    y_logits = model3(x).cpu()
    y_pred = torch.argmax(y_logits, dim=1)
    y_pred = y_pred.cpu().tolist()
    img_ids.extend(img_id)
    y_preds.extend(y_pred)

100%|██████████| 5000/5000 [01:15<00:00, 66.32it/s]


In [33]:
pred_df = pd.DataFrame(list(zip(img_ids, y_preds)), columns=['img_id','a_type'])
label_decoding = {0:'HJ', 1:'HR', 2:'SG', 3:'AR'}
pred_df['a_type'] = pred_df['a_type'].replace(label_decoding)

In [34]:
sample_df = pd.read_csv(SAMPLE_DIR)
sorter = list(sample_df['img_id'])
resdf = pred_df.set_index('img_id')
result = resdf.loc[sorter].reset_index()

In [35]:
result.to_csv(os.path.join('./apple/results','seresnext.csv'),index=False)